# XGBoost Modeling Pipeline
The objective of this notebook is to write production level Python code for modeling cognitive fatigue using the smooth pursuit eye tracking dataset. This code will be reviewed in Code Review 1.

Written by Casey Munk on 6/24/2024

In [15]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split,RandomizedSearchCV, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
import pickle
import os

### Connect to database

In [16]:
PROJ_DIRECTORY = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_DIRECTORY = os.path.join(PROJ_DIRECTORY, 'database')
OUTPUT_DIRECTORY = os.path.join(PROJ_DIRECTORY, 'outputs')
MODEL_DIRECTORY = os.path.join(PROJ_DIRECTORY, 'models')

merged_df = pd.read_csv(os.path.join(DATA_DIRECTORY, 'merged_df.csv'))

### Feature Engineering Functions

In [17]:
def get_dt(final_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the time derivative of the marginal gaze deviation with respect to trial duration in minutes.
    
    Args:
    final_df (pd.DataFrame): Merged, partially preprocessed DataFrame containing the columns 'gazeDeviation_marginal' and 'trialDuration'.
    
    Returns:
    pd.DataFrame: The final DataFrame with an additional 'time_derivative' column.
    """
    final_df['time_derivative'] = None #intializing
    
    for j in range(0, final_df.shape[0] - 1, 2):
        # time derivative calculation for the current and next row
        final_df.loc[j, 'time_derivative'] = (final_df['gazeDeviation_marginal'].iloc[j+1] - final_df['gazeDeviation_marginal'].iloc[j]) / (final_df['trialDuration'].iloc[j] / 60)
        final_df.loc[j+1, 'time_derivative'] = (final_df['gazeDeviation_marginal'].iloc[j+1] - final_df['gazeDeviation_marginal'].iloc[j]) / (final_df['trialDuration'].iloc[j+1] / 60)
    
    # error catching in case the number of rows is odd in test split
    if final_df.shape[0] % 2 != 0:
        last_idx = final_df.shape[0] - 1
        final_df.loc[last_idx, 'time_derivative'] = (final_df['gazeDeviation_marginal'].iloc[last_idx] - final_df['gazeDeviation_marginal'].iloc[last_idx - 1]) / (final_df['trialDuration'].iloc[last_idx - 1] / 60)
    
    return final_df


In [18]:
def get_d_mental(final_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the mental fatigue derivative of the marginal gaze deviation with respect to change in mental fatigue (before and after).
    
    Args:
    final_df (pd.DataFrame): Merged, partially preprocessed DataFrame containing the columns 'gazeDeviation_marginal', 'mentalFatigue', and 'mentalFatigue_before'.
    
    Returns:
    pd.DataFrame: The DataFrame with an additional 'mental_derivative' column.
    """
    final_df['mental_derivative'] = None
    
    for j in range(0, final_df.shape[0] - 1, 2):
        if pd.isna(final_df['mentalFatigue'].iloc[j]) or (final_df['mentalFatigue'].iloc[j] - final_df['mentalFatigue_before'].iloc[j]) == 0: # results in a division by 0
            # if not change in mental fatigue then just take the difference
            final_df.loc[j, 'mental_derivative'] = (final_df['gazeDeviation_marginal'].iloc[j+1] - final_df['gazeDeviation_marginal'].iloc[j])
            final_df.loc[j+1, 'mental_derivative'] = (final_df['gazeDeviation_marginal'].iloc[j+1] - final_df['gazeDeviation_marginal'].iloc[j])
        else:
            # mental fatigue derivative calculation for the current and next row
            final_df.loc[j, 'mental_derivative'] = (final_df['gazeDeviation_marginal'].iloc[j+1] - final_df['gazeDeviation_marginal'].iloc[j]) / (final_df['mentalFatigue'].iloc[j] - final_df['mentalFatigue_before'].iloc[j])
            final_df.loc[j+1, 'mental_derivative'] = (final_df['gazeDeviation_marginal'].iloc[j+1] - final_df['gazeDeviation_marginal'].iloc[j]) / (final_df['mentalFatigue'].iloc[j+1] - final_df['mentalFatigue_before'].iloc[j+1])

    # error catching in case the number of rows is odd in test split
    if final_df.shape[0] % 2 != 0:
        last_idx = final_df.shape[0] - 1
        if pd.isna(final_df['mentalFatigue'].iloc[j]) or (final_df['mentalFatigue'].iloc[last_idx] - final_df['mentalFatigue_before'].iloc[last_idx]) == 0:
            final_df.loc[last_idx, 'mental_derivative'] = (final_df['gazeDeviation_marginal'].iloc[last_idx] - final_df['gazeDeviation_marginal'].iloc[last_idx - 1])
        else:
            final_df.loc[last_idx, 'mental_derivative'] = (final_df['gazeDeviation_marginal'].iloc[last_idx] - final_df['gazeDeviation_marginal'].iloc[last_idx - 1]) / (final_df['mentalFatigue'].iloc[last_idx] - final_df['mentalFatigue_before'].iloc[last_idx])
    
    return final_df

### Preprocessing

In [19]:
### splitting the merged dataset so we can isolate mental fatigue after the trials are complete
before = merged_df[merged_df['timeOfReport'].str.contains('before')].reset_index()
after = merged_df[merged_df['timeOfReport'].str.contains('after')].reset_index()

final_df = before.rename(columns={"mentalFatigue":'mentalFatigue_before', 
                                  "physicalFatigue":'physicalFatigue_before',
                                  "sleepiness":'sleepiness_before'})

#add the mental fatigue outcome column
temp = after[["mentalFatigue", "physicalFatigue", "sleepiness"]].reset_index()
final_df = pd.concat([final_df, temp], axis=1)

# remove duplicate measures from initial merges
final_df = final_df.drop(columns=['index','sleepScore_complete', "caffeineScore_complete", "alertnessScore_complete", 'sleepScore_marginal', "caffeineScore_marginal", "alertnessScore_marginal", 'timeOfReport'])

# recode mental fatigue for binary classification
final_df['mentalFatigue_binary'] = final_df['mentalFatigue'].apply(lambda x: 0 if x in [1, 2] else 1)

# apply feature engineering
get_dt(final_df)
get_d_mental(final_df)

final_df

,subjectID,sessionType,session,sessionTime,trial,gazeDeviation_marginal,angular_velocity_gain,mentalFatigue_before,physicalFatigue_before,sleepiness_before,...,trialDuration,typingSpeed,correctedErrorRate,falseSelections,mentalFatigue,physicalFatigue,sleepiness,mentalFatigue_binary,time_derivative,mental_derivative
0,1,MA,1,M,1,1.654252,0.809592,3,3,2,...,399.293313,1.111965,8.823529,19.178082,2,3,2,0,0.222733,-1.482264
1,1,MA,1,M,2,3.136516,0.775999,3,3,2,...,448.210546,1.606388,18.627451,24.752475,2,3,2,0,0.198424,-1.482264
2,1,MA,2,A,1,2.463788,0.988573,1,2,2,...,311.774936,1.655040,11.111111,11.111111,2,2,2,0,0.120531,0.62631
3,1,MA,2,A,2,3.090098,0.931487,1,2,2,...,181.115151,1.523892,16.666667,23.529412,2,2,2,0,0.207485,0.62631
4,2,MA,1,M,1,2.898291,0.854306,2,2,2,...,373.052926,0.868509,15.000000,21.153846,2,1,2,0,0.027245,0.169394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,26,AM,2,M,2,2.265498,0.872967,1,1,1,...,358.017037,1.106093,11.475410,15.517241,1,1,1,0,0.064365,0.384065
76,27,MA,1,M,1,2.593296,0.983046,1,2,1,...,462.873015,1.140702,23.188406,23.456790,2,1,2,0,0.057472,0.443369
77,27,MA,1,M,2,3.036665,1.050465,1,2,1,...,389.072175,0.956121,20.289855,21.538462,2,1,2,0,0.068373,0.443369
78,27,MA,2,A,1,2.215605,1.043273,3,3,2,...,357.023045,1.008338,15.068493,17.543860,2,1,2,0,0.026741,-0.159118


### Split data

In [20]:
X = final_df[[
                'sessionType', 
                'sessionTime', 
                # 'gazeDeviation_marginal', 
                'angular_velocity_gain', 
                'mentalFatigue_before', 
                'physicalFatigue_before', 
                'sleepiness_before', 
                'trialDifficultyLevel', 
                'gazeDeviation_complete', 
                # 'sleepScore', 
                # 'caffeineScore', 
                # 'alertnessScore', 
                'trialDuration', 
                'typingSpeed', 
                'correctedErrorRate', 
                'falseSelections',
                # 'time_derivative',
                'mental_derivative']]

y = final_df['mentalFatigue_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)


### Save train test split to csv for future demo

In [21]:
# X_test.to_csv(os.path.join(DATA_DIRECTORY, 'X_test.csv'), index=False)

### Preprocessing pipeline (to prevent leakage)

First we need to define column transformers since we are applying separate transformers to different feature subsets (based on if the feature is categorical or quantitative)

In [22]:
# categorical columns transformer - impute nans with the mode and one-hot encode
categorical_features = ['sessionType', 
                        'sessionTime', 
                        'trialDifficultyLevel']

categorical_transformer = Pipeline(steps=[
    ('ohe', OneHotEncoder())
])

# numerical columns transformer - impute nans with the mean and rescale
numerical_features =  [
    # 'gazeDeviation_marginal', 
                        'angular_velocity_gain', 
                        'mentalFatigue_before', 
                       'physicalFatigue_before', 
                       'sleepiness_before', 
                       'gazeDeviation_complete', 
                    #    'time_derivative', 
                       'mental_derivative',
                    #    'sleepScore', 
                    #    'caffeineScore', 
                    #    'alertnessScore', 
                       'trialDuration', 
                       'typingSpeed', 'correctedErrorRate', 'falseSelections']

numerical_transformer = Pipeline(steps=[
    ('rescale', StandardScaler()),
])

# bundle individual transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers = [
        ('categorical_transformer', categorical_transformer, categorical_features),
        ('numerical_transformer', numerical_transformer, numerical_features)
    ],
    # retain cols which are not transformed by the ColumnTransformer 
    remainder='passthrough'
    )

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('categorical_transformer',
                                 Pipeline(steps=[('ohe', OneHotEncoder())]),
                                 ['sessionType', 'sessionTime',
                                  'trialDifficultyLevel']),
                                ('numerical_transformer',
                                 Pipeline(steps=[('rescale',
                                                  StandardScaler())]),
                                 ['angular_velocity_gain',
                                  'mentalFatigue_before',
                                  'physicalFatigue_before', 'sleepiness_before',
                                  'gazeDeviation_complete', 'mental_derivative',
                                  'trialDuration', 'typingSpeed',
                                  'correctedErrorRate', 'falseSelections'])])

In [23]:
pipeline = imbpipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=30)),
    ('classifier', XGBClassifier(random_state=30))
])

In [24]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical_transformer',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder())]),
                                                  ['sessionType', 'sessionTime',
                                                   'trialDifficultyLevel']),
                                                 ('numerical_transformer',
                                                  Pipeline(steps=[('rescale',
                                                                   StandardScaler())]),
                                                  ['angular_velocity_gain',
                                                   'mentalFatigue_before',
                                                   'physicalFatigue_be...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=30, ...))])

### Perform baseline predict with base pipeline- no tuning yet

In [25]:
# fits both the preprocessing steps and the model using the training data
pipeline.fit(X_train, y_train)

# # transform the training and test data using fitted pipeline - only if you want to explore further
X_train_transformed = pipeline.named_steps['preprocessor'].transform(X_train)
X_test_transformed = pipeline.named_steps['preprocessor'].transform(X_test)

# evaluate pipeline on UNSEEN test data
y_pred = pipeline.predict(X_test) # internally transforms X_test before passing to model for prediction
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         6

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16

[[10  0]
 [ 0  6]]


### Perform hyperparameter tuning

In [26]:
param_grid = {
    'classifier__n_estimators': [100, 200, 300, 400, 500],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'classifier__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'classifier__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'classifier__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'classifier__min_child_weight': [1, 3, 5, 7, 10],
    'classifier__gamma': [0, 1, 2, 3, 4, 5]
}

# init kfold
strat_kfold = RepeatedStratifiedKFold(n_splits=5, random_state=30)

random_search = RandomizedSearchCV(estimator=pipeline, 
                                   param_distributions=param_grid, 
                                   n_iter=50, 
                                   scoring='roc_auc', 
                                   cv=strat_kfold, 
                                   verbose=4, 
                                   n_jobs=-1, 
                                   random_state=30)

random_search.fit(X_train, y_train)

# for the randomized search
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best ROC AUC Score: {random_search.best_score_}")


Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Best Parameters: {'classifier__subsample': 0.7, 'classifier__n_estimators': 400, 'classifier__min_child_weight': 1, 'classifier__max_depth': 7, 'classifier__learning_rate': 0.2, 'classifier__gamma': 1, 'classifier__colsample_bytree': 0.6}
Best ROC AUC Score: 0.9844583333333332


### Perform nested cross-validation for robust evaluations

In [27]:
# set up scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score, average='weighted'),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
    'roc_auc': make_scorer(roc_auc_score)
}

# perform nested cross-validation - this redoes hyperparameter tuning for every fold 
cv_results = cross_validate(random_search, X, y, cv=strat_kfold, scoring=scoring, return_train_score=False)

# calculate and print the mean, min, and max of each metric
metrics = ['test_accuracy', 'test_precision_weighted', 'test_recall_weighted', 'test_f1_weighted', 'test_roc_auc']

for metric in metrics:
    mean_score = np.mean(cv_results[metric])
    min_score = np.min(cv_results[metric])
    max_score = np.max(cv_results[metric])
    print(f'{metric}: Mean = {mean_score:.4f}, Min = {min_score:.4f}, Max = {max_score:.4f}')

Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidates, totalling 2500 fits
Fitting 50 folds for each of 50 candidat

### Make predictions from best model

In [28]:
# predict using the best model on the test data
y_pred = random_search.best_estimator_.predict(X_test)
# show all eval metrics
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         6

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16

[[10  0]
 [ 0  6]]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
ROC AUC Score: 1.0000


### Save the best model (pipelined model) to a unique object

In [92]:
best_pipeline = random_search.best_estimator_

## Save the best pipelined model (pickling)

In [93]:
# out_pkl = os.path.join(MODEL_DIRECTORY, 'cf_xgboost_pipeline.pkl')
# pickle.dump(best_pipeline, open(out_pkl, 'wb'))